## Building a Conversational Agent with Function Calling

* **Created by:** Eric Martinez
* **For:** CSCI 3351
* **At:** University of Texas Rio-Grande Valley

## Setup

In [2]:
%pip install -U --quiet pydantic openai gradio geocoder

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vcrpy 5.1.0 requires urllib3<2; python_version < "3.10", but you have urllib3 2.2.1 which is incompatible.
kubernetes 28.1.0 requires urllib3<2.0,>=1.24.2, but you have urllib3 2.2.1 which is incompatible.
You should consider upgrading via the 'c:\users\kty325\.pyenv\pyenv-win\versions\3.9.6\python.exe -m pip install --upgrade pip' command.


## Database

In [4]:
from datetime import datetime
from datetime import timedelta
import sqlite3

def create_database():
    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('schedule.db')
    cursor = conn.cursor()

    # Create tables
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
        full_name TEXT NOT NULL,
        phone TEXT NOT NULL UNIQUE
    );
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS pickup_slots (
        slot_id INTEGER PRIMARY KEY AUTOINCREMENT,
        pickup_date TEXT NOT NULL,
        pickup_time TEXT NOT NULL
    );
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS scheduled_pickups (
        pickup_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        slot_id INTEGER,
        wash_and_fold_service BOOLEAN,
        ironing_service BOOLEAN,
        FOREIGN KEY(user_id) REFERENCES users(user_id),
        FOREIGN KEY(slot_id) REFERENCES pickup_slots(slot_id)
    );
    ''')

    # Commit changes
    conn.commit()
    
    # Generate and insert pickup slots
    generate_pickup_slots(cursor)
    
    # Commit changes and close the connection
    conn.commit()
    conn.close()

    print("Database and tables created successfully, and initial data inserted.")

def generate_pickup_slots(cursor):
    # Define the hours and days you want to create slots for
    hours = ["08:00am", "09:00am", "10:00am", "11:00am", "12:00pm", "01:00pm", "02:00pm", "03:00pm", "04:00pm", "05:00pm"]
    days_of_week = [0, 1, 2, 3, 4, 5, 6]  # 0=Monday, 1=Tuesday, ..., 6=Sunday

    start_date = datetime.now()
    end_date = start_date + timedelta(days=365)  # Slots for the next year

    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() in days_of_week:
            for hour in hours:
                pickup_date = current_date.strftime("%m/%d/%Y")
                cursor.execute('INSERT INTO pickup_slots (pickup_date, pickup_time) VALUES (?, ?)', (pickup_date, hour))
        current_date += timedelta(days=1)
        
create_database()

Database and tables created successfully, and initial data inserted.


In [5]:
import sqlite3
import re
import geocoder

def connect_db():
    return sqlite3.connect('schedule.db')

def format_phone_number(phone):
    # Remove all non-digit characters
    digits = re.sub(r'\D', '', phone)

    # Check if we have exactly 10 digits (U.S. phone number without country code)
    if len(digits) == 10:
        # Reformat to (XXX) XXX-XXXX
        formatted_phone = f"({digits[0:3]}) {digits[3:6]}-{digits[6:10]}"
        return True, formatted_phone
    return False, None

def validate_phone_number(phone):
    pattern = re.compile(r"^\(\d{3}\) \d{3}-\d{4}$")
    return pattern.match(phone) is not None

def validate_date(date_text):
    try:
        datetime.strptime(date_text, "%m/%d/%Y")
        return True
    except ValueError:
        return False

def validate_address(address):
    # Use the geocoder to get the location data
    g = geocoder.osm(address)
    print(g)
    print(g.city)
    if g.ok:
        # Check if the address is in Edinburg, TX
        if 'Edinburg' in g.city and 'Texas' in g.state:
            return True, g.latlng
        else:
            return False, None
    return False, None


def is_slot_available(slot_id):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('SELECT slot_id FROM scheduled_pickups WHERE slot_id = ?', (slot_id,))
    result = cursor.fetchone()
    conn.close()
    return result is None

def schedule_pickup(full_name: str, phone: str, address: str, pickup_slot_id: int, wash_and_fold_service: bool, ironing_service: bool):
    "Attempts to schedule a pickup for a user at the desired pickup_slot_id. All fields are required."

    valid_phone, formatted_phone = format_phone_number(phone)
    if not valid_phone:
        return "Invalid phone number. Please provide a valid 10-digit phone number."

    phone = formatted_phone  # Use the formatted phone number

    if not is_slot_available(pickup_slot_id):
        return "This slot is already booked. Please choose another slot."

    valid_address, coordinates = validate_address(address)
    if not valid_address:
        return "Invalid address or not located in Edinburg, TX."

    conn = connect_db()
    cursor = conn.cursor()
    
    try:
        # Insert user if not exists
        cursor.execute('INSERT OR IGNORE INTO users (full_name, phone) VALUES (?, ?)', (full_name, phone))
        user_id = cursor.lastrowid
        
        if user_id == 0:
            cursor.execute('SELECT user_id FROM users WHERE phone = ?', (phone,))
            user_id = cursor.fetchone()[0]

        # Schedule the pickup
        cursor.execute('''
            INSERT INTO scheduled_pickups (user_id, slot_id, wash_and_fold_service, ironing_service)
            VALUES (?, ?, ?, ?)
        ''', (user_id, pickup_slot_id, wash_and_fold_service, ironing_service))
        
        conn.commit()
    except sqlite3.Error as e:
        conn.rollback()
        conn.close()
        return f"An error occurred: {e}"
    finally:
        conn.close()

    return "Successfully scheduled."

def get_availability_admin():
    "Returns all availability"
    conn = connect_db()
    cursor = conn.cursor()
    
    cursor.execute('SELECT slot_id, pickup_date, pickup_time FROM pickup_slots')
    availability = cursor.fetchall()
    
    conn.close()
    
    return [{"pickup_slot_id": slot_id, "pickup_date": date, "pickup_time": time} for slot_id, date, time in availability]

def get_booked_appointments():
    "Attempts to schedule a pickup for a user at the desired pickup_slot_id."
    conn = connect_db()
    cursor = conn.cursor()
    
    # SQL query to fetch all booked appointments
    cursor.execute('''
        SELECT sp.pickup_id, u.full_name, u.phone, ps.pickup_date, ps.pickup_time, sp.wash_and_fold_service, sp.ironing_service
        FROM scheduled_pickups sp
        JOIN users u ON sp.user_id = u.user_id
        JOIN pickup_slots ps ON sp.slot_id = ps.slot_id
        ORDER BY ps.pickup_date, ps.pickup_time
    ''')
    
    # Fetch all results
    appointments = cursor.fetchall()
    conn.close()
    
    # Convert to a list of dictionaries for easier DataFrame conversion
    columns = ["pickup_id", "full_name", "phone", "pickup_date", "pickup_time", "wash_and_fold_service", "ironing_service"]
    appointments_data = [dict(zip(columns, appointment)) for appointment in appointments]
    
    return appointments_data


def get_availability():
    "Returns available slots for today and tomorrow"
    conn = connect_db()
    cursor = conn.cursor()
    
    # Get today's and tomorrow's dates in the format stored in the database
    today = datetime.now().strftime("%m/%d/%Y")
    tomorrow = (datetime.now() + timedelta(days=1)).strftime("%m/%d/%Y")
    
    # SQL query to fetch slots for today and tomorrow that are not booked
    cursor.execute('''
        SELECT ps.slot_id, ps.pickup_date, ps.pickup_time
        FROM pickup_slots ps
        LEFT JOIN scheduled_pickups sp ON ps.slot_id = sp.slot_id
        WHERE ps.pickup_date IN (?, ?) AND sp.slot_id IS NULL
        ORDER BY ps.pickup_date, ps.pickup_time
    ''', (today, tomorrow))
    
    availability = cursor.fetchall()
    conn.close()
    
    # Format the results into a list of dictionaries
    return [{"pickup_slot_id": slot_id, "pickup_date": date, "pickup_time": time} for slot_id, date, time in availability]

# Example usage
print(get_booked_appointments())

[]


## Define Tools

In [6]:
def sum(a:int, b:int=1):
    "Adds a + b"
    return a + b

print(sum(a=2, b=3))

5


In [7]:
import requests

def geocode(city_name:str):
    "Geocodes a city name into latitude and longitude data"
    url = f'https://geocoding-api.open-meteo.com/v1/search?name={city_name}&count=10&language=en&format=json'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response
    
print(geocode(city_name="Edinburg"))

{'results': [{'id': 4688275, 'name': 'Edinburg', 'latitude': 26.30174, 'longitude': -98.16334, 'elevation': 30.0, 'feature_code': 'PPLA2', 'country_code': 'US', 'admin1_id': 4736286, 'admin2_id': 4697444, 'timezone': 'America/Chicago', 'population': 84497, 'postcodes': ['78539', '78540'], 'country_id': 6252001, 'country': 'United States', 'admin1': 'Texas', 'admin2': 'Hidalgo'}, {'id': 4257043, 'name': 'Edinburgh', 'latitude': 39.35422, 'longitude': -85.96666, 'elevation': 205.0, 'feature_code': 'PPL', 'country_code': 'US', 'admin1_id': 4921868, 'admin2_id': 4259727, 'admin3_id': 4254728, 'timezone': 'America/Indiana/Indianapolis', 'population': 4546, 'postcodes': ['46124'], 'country_id': 6252001, 'country': 'United States', 'admin1': 'Indiana', 'admin2': 'Johnson', 'admin3': 'Blue River Township'}, {'id': 4757240, 'name': 'Edinburg', 'latitude': 38.82095, 'longitude': -78.56585, 'elevation': 246.0, 'feature_code': 'PPL', 'country_code': 'US', 'admin1_id': 6254928, 'admin2_id': 4785252

In [8]:
def weather(latitude:float, longitude:float):
    "Returns the weather conditions for a given latitude and longitude"
    url = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,is_day,precipitation,rain,showers,snowfall&timezone=America%2FChicago'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response
    
print(weather(latitude=26.30174, longitude=-98.16334))

{'latitude': 26.309208, 'longitude': -98.153625, 'generationtime_ms': 0.8040666580200195, 'utc_offset_seconds': -18000, 'timezone': 'America/Chicago', 'timezone_abbreviation': 'CDT', 'elevation': 30.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C', 'is_day': '', 'precipitation': 'mm', 'rain': 'mm', 'showers': 'mm', 'snowfall': 'cm'}, 'current': {'time': '2024-04-22T16:15', 'interval': 900, 'temperature_2m': 19.5, 'is_day': 1, 'precipitation': 0.0, 'rain': 0.0, 'showers': 0.0, 'snowfall': 0.0}}


In [9]:
from datetime import datetime
from datetime import timedelta

# gets todays date
def get_todays_date():
    "Gets today's date"
    return datetime.now().strftime('%B %d, %Y')

# Example:
print(get_todays_date())

April 22, 2024


In [10]:
# gets tomorrows date
def get_tomorrows_date():
    "Gets tomorrow's date"
    now = datetime.now()
    tomorrow = now + timedelta(days=1)
    return tomorrow.strftime('%B %d, %Y')

print(get_tomorrows_date())

April 23, 2024


In [11]:
# gets the current time
def get_current_time():
    "Gets the current time"
    return datetime.now().strftime("%m/%d/%Y %I:%M %p")

# Example:
print(get_current_time())

04/22/2024 04:28 PM


In [12]:
import geocoder

# finds your city and state based on your IP address
def user_location():
    "Guesses the users current location in the format of <city>, <state> based on IP address. But should always ask the user when scheduling."
    g = geocoder.ip('me')
    city = g.city
    state = g.state
    return f"{city}, {state}"

# Example:
print(user_location())

Edinburg, Texas


## Format for OpenAI

In [13]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

def schema(f):
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
          for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).model_json_schema()
    function_schema = dict(name=f.__name__, description=f.__doc__, parameters=s)
    return {
        "type": "function",
        "function": function_schema
    }

def call_func(name, arguments):
    f = globals()[name]
    return f(**json.loads(arguments))

In [14]:
schema(sum)

{'type': 'function',
 'function': {'name': 'sum',
  'description': 'Adds a + b',
  'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
    'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
   'required': ['a'],
   'title': 'Input for `sum`',
   'type': 'object'}}}

In [15]:
call_func("sum", '{"a": 2, "b": 3}')

5

In [16]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)  # take environment variables from .env.

from openai import OpenAI

client = OpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY")
)

def chat_completion(
    message,
    model="gpt-4",
    prompt="You are a helpful assistant.",
    temperature=0,
    messages=[],
    tools=[]
):
    # Add the prompt to the messages list
    if prompt is not None:
        messages = [{"role": "system", "content": prompt}] + messages

    if message is not None:
        # Add the user's message to the messages list
        messages += [{"role": "user", "content": message}]

    # NEW
    if tools:
        tool_schemas = [schema(f) for f in tools]
    else:
        tool_schemas = None
        
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    print(f"messages: {messages}")
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        tools=tool_schemas
    )
    
    # Updated
    assistant_message = completion.choices[0].message
        
    return assistant_message

In [17]:
chat_completion("Add 2+3", model="gpt-4", tools=[sum])

messages: [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Add 2+3'}]


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_IxgubY6rJwQMnqlYMsRXLSF2', function=Function(arguments='{\n  "a": 2,\n  "b": 3\n}', name='sum'), type='function')])

In [18]:
# Define a function to handle the chat interaction with the AI model
def chat(message, chatbot_messages, history_state):
    prompt = """
    You are a helpful AI assistant named Alfred created by SpinPlus Laundromat in Edinburg, TX.

    Today's date: {todays_date}
    Current time: {current_time}
    
    ## Task
    
    Your job is to provide an amazing customer service experience for potential customers of SpinPlus and current customers.
    
    Additionally, you are renowned for converting leads into customers.
    
    As an AI assistant, you will help faciliate 'pick-ups' for SpinPlus Laundromat's services.
    
    ## Pick-Up Services
    
    Wash and Fold - $1.25 per lb - 15 pound minimum
    Ironing - $6 per shirt, pants, etc.
    
    An additional $10 charge is added for pick-ups on top of the standard pricing.
    
    ## Turn around time
    
    Any Wash and Fold picked up before 10am is guaranteed to be done by the next business day.
    
    Ironing may take longer based on staff availability and demand. Account for 2-3 days (minimum).
        
    ## Delivery
    
    SpinPlus Laundromat does NOT do delivery. Customers must pick-up items at the store location.
    
    ## Prices
    
    Prices for standard services will be determined by SpinPlus Laundromat staff when they arrive to your location.
    They will weigh Wash and Fold items and count items that need to be ironed on top of adding the Pickup Service charge.
    
    ## Appointments
    
    If a customer expresses intent for an appointment or a pickup what they mean is that they would like to schedule a pick-up for the Pick-Up Services.
        
    Before getting down any of their details, you should always check availability and provide it to the customer.
    
    They don't have a user interface or anything to indicate availability and are relying on you to tell them.
    
    When displaying availability do not display slot IDs as that maps to our internal database IDs and is SENSITIVE.
    
    ## Pickup Scheduling Process
    
    Incrementally ask the customer for the correct information
    
    1. What service(s) they are interested in
    2. Show them the available appointment slots and identify the one they are interested in
    3. What their name and phone number is
    4. What the pickup location is. If they don't specify their city, state assume it is Edinburg, TX
    5. Schedule the appointment.
    
    It is important that you break this up into smaller interactions to not overwhelm the user.
    
    Getting their name and phone number before the pickup location is important in case the session gets disconnected.
    """.format(
        todays_date=get_todays_date(),
        current_time=get_current_time()
    )
    
    # Initialize chatbot_messages and history_state if they are not provided
    allowed_tools = [sum, weather, get_tomorrows_date, user_location, schedule_pickup, get_availability]
    chatbot_messages = chatbot_messages or []
    history_state = history_state or []
    
    # Try to get the AI's reply using the chat_completion
    try:
        assistant_message = chat_completion(message, model="gpt-4", prompt=prompt, messages=history_state, tools=allowed_tools)
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
        
    # UPDATED
    while(assistant_message.tool_calls):
        for tool_call in assistant_message.tool_calls:
            tool_call_json = json.dumps({ "name": tool_call.function.name, "arguments": tool_call.function.arguments})
            history_state.append({"role": assistant_message.role, "content": tool_call_json})
            
        for tool_call in assistant_message.tool_calls:
            results = call_func(tool_call.function.name, tool_call.function.arguments)
            results_str = str(results) # convert to string
            history_state.append({"role": "function", "tool_call_id": tool_call.id, "name": tool_call.function.name, "content": results_str})

        # Try to get the AI's reply using the get_ai_reply function
        try:
            assistant_message = chat_completion(None, model="gpt-4", prompt=prompt, messages=history_state, tools=allowed_tools)
        except Exception as e:
            # If an error occurs, raise a Gradio error
            raise gr.Error(e)
        
    # Append the user's message and the AI's reply to the chatbot_messages list
    if(assistant_message.content):
        chatbot_messages.append((message, assistant_message.content.strip()))
    else:
        chatbot_messages.append((message, None))
    
    # Append the user's message and the AI's reply to the history_state list
    history_state.append({"role": "user", "content": message})
    history_state.append({"role": "assistant", "content": assistant_message.content})
    
    # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    return None, chatbot_messages, history_state

In [19]:
import gradio as gr
import pandas as pd

def get_availability_df():
    # Get availability data as a list of dictionaries
    availability_data = get_availability()
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(availability_data)
    
    return df 

def get_booked_appointments_df():
    # Get booked appointments data as a list of dictionaries
    appointments_data = get_booked_appointments()
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(appointments_data)
    
    return df

# Define a function to return a chatbot app using Gradio
def get_chatbot_app(share=False):
    # Create the Gradio interface using the Blocks layout
    with gr.Blocks() as app:
        with gr.Tab("Conversation"):
            with gr.Row():
                with gr.Column():
                    # Create a chatbot interface for the conversation
                    chatbot = gr.Chatbot(label="Conversation")
                    # Create a textbox for the user's message
                    message = gr.Textbox(label="Message")
                    # Create a state object to store the conversation history
                    history_state = gr.State()
                    # Create a state object to store the available tools
                    tools = gr.State()
                    # Create a button to send the user's message
                    btn = gr.Button(value="Send")
                btn.click(chat, inputs=[message, chatbot, history_state], outputs=[message, chatbot, history_state])
        with gr.Tab("(Admin) Appointments"):
            refresh = gr.Button(value="Refresh")
            appointments = gr.Dataframe(value=get_booked_appointments_df())
            refresh.click(get_booked_appointments_df, outputs=appointments)
        with gr.Tab("(Admin) Update Appointment"):
            appointment_slots = gr.Dataframe(value=get_availability_df())
        # Return the app
        return app
    
# Call the launch_chatbot function to start the chatbot interface using Gradio
# Set the share parameter to False, meaning the interface will not be publicly accessible
get_chatbot_app().launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Alfred created by SpinPlus Laundromat in Edinburg, TX.\n\n    Today's date: April 22, 2024\n    Current time: 04/22/2024 04:32 PM\n    \n    ## Task\n    \n    Your job is to provide an amazing customer service experience for potential customers of SpinPlus and current customers.\n    \n    Additionally, you are renowned for converting leads into customers.\n    \n    As an AI assistant, you will help faciliate 'pick-ups' for SpinPlus Laundromat's services.\n    \n    ## Pick-Up Services\n    \n    Wash and Fold - $1.25 per lb - 15 pound minimum\n    Ironing - $6 per shirt, pants, etc.\n    \n    An additional $10 charge is added for pick-ups on top of the standard pricing.\n    \n    ## Turn around time\n    \n    Any Wash and Fold picked up before 10am is guaranteed to be done by the next business day.\n    \n    Ironing may take longer based on staff availability and demand. Account for 2-3 days (mi

messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Alfred created by SpinPlus Laundromat in Edinburg, TX.\n\n    Today's date: April 22, 2024\n    Current time: 04/22/2024 04:32 PM\n    \n    ## Task\n    \n    Your job is to provide an amazing customer service experience for potential customers of SpinPlus and current customers.\n    \n    Additionally, you are renowned for converting leads into customers.\n    \n    As an AI assistant, you will help faciliate 'pick-ups' for SpinPlus Laundromat's services.\n    \n    ## Pick-Up Services\n    \n    Wash and Fold - $1.25 per lb - 15 pound minimum\n    Ironing - $6 per shirt, pants, etc.\n    \n    An additional $10 charge is added for pick-ups on top of the standard pricing.\n    \n    ## Turn around time\n    \n    Any Wash and Fold picked up before 10am is guaranteed to be done by the next business day.\n    \n    Ironing may take longer based on staff availability and demand. Account for 2-3 days (mi

messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Alfred created by SpinPlus Laundromat in Edinburg, TX.\n\n    Today's date: April 22, 2024\n    Current time: 04/22/2024 04:34 PM\n    \n    ## Task\n    \n    Your job is to provide an amazing customer service experience for potential customers of SpinPlus and current customers.\n    \n    Additionally, you are renowned for converting leads into customers.\n    \n    As an AI assistant, you will help faciliate 'pick-ups' for SpinPlus Laundromat's services.\n    \n    ## Pick-Up Services\n    \n    Wash and Fold - $1.25 per lb - 15 pound minimum\n    Ironing - $6 per shirt, pants, etc.\n    \n    An additional $10 charge is added for pick-ups on top of the standard pricing.\n    \n    ## Turn around time\n    \n    Any Wash and Fold picked up before 10am is guaranteed to be done by the next business day.\n    \n    Ironing may take longer based on staff availability and demand. Account for 2-3 days (mi